In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
apikey = 'qhlsyTscFtclLd2FC9fWqKJh2yCYm9Vd1C8sowvzK8I4SpIlzm9n8DtXLWpVOWyI'
secret = 'KF5OLsWZUUWk2tZ5AwICQFAkUZYM1gcIqd9ceT6gyptUAP7mj6H2rzCX3W9UQ5ni'

In [3]:
client = Client(apikey, secret)

In [16]:
crypto_save = {}

tickers=['BTCUSDT','LTCUSDT','XRPUSDT','BCHUSDT','XMRUSDT','BCHUSDT','XRPUSDT','DASHUSDT','XLMUSDT','NEOUSDT','ETCUSDT',"DOGEUSDT",'ZECUSDT']



start_date = '2010-01-1'
end_date = '2026-01-5'
interval = '15m'

for ticker in tickers:
    klines = client.get_historical_klines(ticker, interval, start_date, end_date)
    df = pd.DataFrame(klines, columns=[
                                        'open_time','open', 'high', 'low', 'close',
                                            'volume','close_time', 'dv','num_trades',
                                            'taker_buy_vol','taker_buy_base_vol', 'ignore'
                                    ])


    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    df.set_index('open_time', inplace=True)
    df.drop(columns=['close_time','ignore'], inplace=True)

    df.index = df.index.round('min')
    df = df.astype(float)


    df = df.drop(columns=['dv','num_trades','taker_buy_vol','taker_buy_base_vol'])
    df.index = pd.to_datetime(df.index)
    df.index.names = ['Datetime']
    full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="15T")  # '5T' = 5 minutes
    df = df.reindex(full_index).ffill()
    df.to_csv(f'{ticker}.csv')


df.head()

C:\Users\Jordi\AppData\Local\Temp\ipykernel_5052\2294036998.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="15T")  # '5T' = 5 minutes
C:\Users\Jordi\AppData\Local\Temp\ipykernel_5052\2294036998.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="15T")  # '5T' = 5 minutes
C:\Users\Jordi\AppData\Local\Temp\ipykernel_5052\2294036998.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="15T")  # '5T' = 5 minutes
C:\Users\Jordi\AppData\Local\Temp\ipykernel_5052\2294036998.py:31: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_

,open,high,low,close,volume
2019-03-21 04:00:00,58.50,58.50,57.08,57.36,298.90386
2019-03-21 04:15:00,57.28,57.99,57.20,57.42,97.46465
2019-03-21 04:30:00,57.20,57.20,57.20,57.20,5.99200
2019-03-21 04:45:00,57.56,57.56,57.20,57.20,43.12151
2019-03-21 05:00:00,57.20,57.20,56.80,56.80,212.46891


In [5]:
print(df.shape)
df_save = df.copy()

(264260, 9)


In [6]:
# Venir bien structurer mon csv -> ohlcv
# Avec un Datetime en index 

df = df.drop(columns=['dv','num_trades','taker_buy_vol','taker_buy_base_vol'])
df.index = pd.to_datetime(df.index)
df.index.names = ['Datetime']
print(df.head())

                        open     high      low    close     volume
Datetime                                                          
2017-08-17 04:00:00  4261.48  4280.56  4261.48  4261.48   2.189061
2017-08-17 04:15:00  4261.48  4270.41  4261.32  4261.45   9.119865
2017-08-17 04:30:00  4280.00  4310.07  4267.99  4310.07  21.923552
2017-08-17 04:45:00  4310.07  4313.62  4291.37  4308.83  13.948531
2017-08-17 05:00:00  4308.83  4328.69  4304.31  4304.31   5.101153


In [7]:
#Verification que y'est aucun index de dupliquer et que le format Datetime et le meme pour chaque ligne

print(df.index.duplicated().sum())
index_duplicate = df.index[df.index.duplicated(keep=False)]
print(index_duplicate)
formats_uniques = df.index.strftime('%Y-%m-%d %H:%M:%S').nunique()
print("Nombre de formats uniques :", formats_uniques)


0
DatetimeIndex([], dtype='datetime64[ns]', name='Datetime', freq=None)
Nombre de formats uniques : 264260


In [8]:
print(df.isna().sum())
print("Valeurs Zéro:\n", (df == 0).sum())
print("Valeurs Négatives:\n", (df < 0).sum())

open      0
high      0
low       0
close     0
volume    0
dtype: int64
Valeurs Zéro:
 open       0
high       0
low        0
close      0
volume    56
dtype: int64
Valeurs Négatives:
 open      0
high      0
low       0
close     0
volume    0
dtype: int64


In [9]:
expected_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='1T')
print(expected_index)
missing_times = expected_index.difference(df.index)


C:\Users\Jordi\AppData\Local\Temp\ipykernel_5052\2956369126.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  expected_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq='1T')


DatetimeIndex(['2017-08-17 04:00:00', '2017-08-17 04:01:00',
               '2017-08-17 04:02:00', '2017-08-17 04:03:00',
               '2017-08-17 04:04:00', '2017-08-17 04:05:00',
               '2017-08-17 04:06:00', '2017-08-17 04:07:00',
               '2017-08-17 04:08:00', '2017-08-17 04:09:00',
               ...
               '2025-03-06 17:51:00', '2025-03-06 17:52:00',
               '2025-03-06 17:53:00', '2025-03-06 17:54:00',
               '2025-03-06 17:55:00', '2025-03-06 17:56:00',
               '2025-03-06 17:57:00', '2025-03-06 17:58:00',
               '2025-03-06 17:59:00', '2025-03-06 18:00:00'],
              dtype='datetime64[ns]', length=3972361, freq='min')


In [13]:
delta = df.index.to_series().diff().dropna().unique()

In [14]:
print(delta)

<TimedeltaArray>
['0 days 00:01:00']
Length: 1, dtype: timedelta64[ns]


In [12]:
full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="1T")  # '5T' = 5 minutes
df = df.reindex(full_index).ffill()

C:\Users\Jordi\AppData\Local\Temp\ipykernel_5052\4163235366.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=df.index.min(), end=df.index.max(), freq="1T")  # '5T' = 5 minutes


In [15]:
df.to_csv('BTCUSDT.csv')